<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n70_x2_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n70_x2_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    16.947452  33.384041  ...  288.487160   232.045802
     1.0    43.539506  36.678309  ...  510.647173   701.407568
     2.0    31.206524  36.770959  ...  421.578753 -1550.760686
     3.0    39.968994  37.082940  ...  483.457422   808.642443
     4.0    44.393599  24.520593  ...  431.478606 -7521.738646
...               ...        ...  ...         ...          ...
99.0 65.0   28.379284  23.686331  ...  353.355578  -503.019483
     66.0   29.290968  14.109050  ...  284.122317   464.196280
     67.0   41.316034  35.076616  ...  492.274538   944.817006
     68.0   45.884959  11.444635  ...  437.708137 -3585.541306
     69.0   12.011950  45.289955  ...  303.294710   -76.519558

[7000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    16.947452  33.384041  ...  288.487160   232.045802
     1.0    43.539506  36.678309  ...  510.647173   701.407568
     2.0    31.206524  36.770959  ...  421.578753 -1550.760686
     3.0    39.968994  37.082940  ...  483.457422   808.642443
     4.0    44.393599  24.520593  ...  431.478606 -7521.738646
...               ...        ...  ...         ...          ...
99.0 65.0   28.379284  23.686331  ...  353.355578  -503.019483
     66.0   29.290968  14.109050  ...  284.122317   464.196280
     67.0   41.316034  35.076616  ...  492.274538   944.817006
     68.0   45.884959  11.444635  ...  437.708137 -3585.541306
     69.0   12.011950  45.289955  ...  303.294710   -76.519558

[7000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,16.947452,33.384041,42.716608,269.957789,520.532962,288.487160,232.045802
1,0.0,1.0,43.539506,36.678309,38.662533,696.318836,1212.054741,510.647173,701.407568
2,0.0,2.0,31.206524,36.770959,45.355513,-1528.373650,-1129.181933,421.578753,-1550.760686
3,0.0,3.0,39.968994,37.082940,38.779989,811.546992,1292.099865,483.457422,808.642443
4,0.0,4.0,44.393599,24.520593,14.352319,-7578.068946,-7090.260040,431.478606,-7521.738646
...,...,...,...,...,...,...,...,...,...
6995,99.0,65.0,28.379284,23.686331,49.101605,-511.244682,-149.663904,353.355578,-503.019483
6996,99.0,66.0,29.290968,14.109050,24.391148,411.908718,748.318597,284.122317,464.196280
6997,99.0,67.0,41.316034,35.076616,42.408190,941.446400,1437.091544,492.274538,944.817006
6998,99.0,68.0,45.884959,11.444635,42.971339,-3666.098733,-3147.833169,437.708137,-3585.541306


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    16.947452  33.384041  ...  288.487160   232.045802
     1.0    43.539506  36.678309  ...  510.647173   701.407568
     2.0    31.206524  36.770959  ...  421.578753 -1550.760686
     3.0    39.968994  37.082940  ...  483.457422   808.642443
     4.0    44.393599  24.520593  ...  431.478606 -7521.738646
...               ...        ...  ...         ...          ...
99.0 65.0   28.379284  23.686331  ...  353.355578  -503.019483
     66.0   29.290968  14.109050  ...  284.122317   464.196280
     67.0   41.316034  35.076616  ...  492.274538   944.817006
     68.0   45.884959  11.444635  ...  437.708137 -3585.541306
     69.0   12.011950  45.289955  ...  303.294710   -76.519558

[7000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
5.0,34.0,30.0,16724681.656869559,44922299.37814644,506808.5350566533,1549044.8061429807,3.0441200819102514,0.0020412117773094263,0.9989793941113453,Reject005=0 : Heteroscedasticity
7.0,34.0,30.0,25990344.00982664,25839736.004941355,787586.1821159589,891025.3794807364,1.1267659299877375,0.7323351425507303,0.6338324287246349,Reject005=1 : Homoscedasticity
8.0,37.0,27.0,50753766.07576355,46406145.33291003,1409826.8354378764,1784851.7435734628,1.2529830096231815,0.5177276536051836,0.7411361731974082,Reject005=1 : Homoscedasticity
9.0,29.0,35.0,5809251.056303491,97884319.6236457,207473.25201083897,2878950.5771660497,13.961205972894234,1.6783507916784401e-10,0.9999999999160825,Reject005=0 : Heteroscedasticity
11.0,31.0,33.0,8187599.432959748,65062081.838332556,272919.9810986583,2033190.0574478924,7.4648162582604645,1.9543797247223438e-07,0.9999999022810138,Reject005=0 : Heteroscedasticity
12.0,30.0,34.0,8099547.244277249,81507616.28589274,279294.73256128444,2469927.7662391737,8.879321619977562,3.215481414464705e-08,0.9999999839225929,Reject005=0 : Heteroscedasticity
13.0,34.0,30.0,32329736.885608528,37138479.04433099,979688.9965335918,1280637.2084252066,1.3019059325836375,0.4548610265671591,0.7725694867164205,Reject005=1 : Homoscedasticity
14.0,30.0,34.0,30090394.940667022,46776358.29894546,1037599.8255402421,1417465.402998347,1.371642260062806,0.38265617468931734,0.8086719126553413,Reject005=1 : Homoscedasticity
16.0,33.0,31.0,8389607.0786019,100714074.52145599,262175.2212063094,3357135.8173818663,12.779115367881229,7.995959450113332e-11,0.9999999999600202,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,33.0,31.0,66340310.49488472,51465261.59598415,2073134.7029651476,1715508.7198661384,0.8258267205127705,0.5946125705049217,Reject005=1 : Homoscedasticity
1.0,32.0,32.0,69250101.54683653,40943873.941123866,2233874.2434463394,1320770.1271330279,0.5912464101360478,0.14249951443987077,Reject005=1 : Homoscedasticity
2.0,30.0,34.0,47138696.98612826,23607728.400598366,1625472.309866492,715385.7091090414,0.441894874499673,0.022421212173664693,Reject005=0 : Heteroscedasticity
3.0,27.0,37.0,91046375.94432923,16814790.149658028,3501783.690166509,467077.50415716745,0.13476925516370672,5.50938338649708e-08,Reject005=0 : Heteroscedasticity
4.0,30.0,34.0,28243482.07732119,24300017.551241677,973913.175080041,736364.1682194448,0.7591554008205912,0.43527513751788033,Reject005=1 : Homoscedasticity
5.0,34.0,30.0,16724681.656869559,44922299.37814644,506808.5350566533,1549044.8061429807,3.0441200819102514,0.0020412117773094263,Reject005=0 : Heteroscedasticity
6.0,33.0,31.0,56507225.9944592,22422246.084874846,1765850.81232685,747408.2028291615,0.4224033685323737,0.017889905273547422,Reject005=0 : Heteroscedasticity
7.0,34.0,30.0,25990344.00982664,25839736.004941355,787586.1821159589,891025.3794807364,1.1267659299877375,1.2676648574492697,Reject005=1 : Homoscedasticity
8.0,37.0,27.0,50753766.07576355,46406145.33291003,1409826.8354378764,1784851.7435734628,1.2529830096231815,1.4822723463948164,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    63
Reject005=1 : Homoscedasticity      37
Name: Result_test, dtype: int64